In [2]:
!pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached Pillow-8.1.0-cp38-cp38-win_amd64.whl (2.2 MB)


In [15]:
import torch
import numpy as np

In [8]:
x = torch.tensor(3.)
w = torch.tensor(4.,requires_grad=True)
b = torch.tensor(5.,requires_grad=True)

In [9]:
y = w*x +b

In [10]:
y

tensor(17., grad_fn=<AddBackward0>)

In [11]:
y.backward()

In [13]:
print("dy/dw ",w.grad)
print("dy/db ",b.grad)

dy/dw  tensor(3.)
dy/db  tensor(1.)


## Training Data

Training data for yeilding amount of apples and oranges related to environmental changes

In [16]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [17]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [19]:
inputs_tensor = torch.from_numpy(inputs)
targets_tensor = torch.from_numpy(targets)

print(inputs_tensor)
print(targets_tensor)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [21]:
# Initializing random weights and bias

w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 1.4280,  1.6965,  1.1977],
        [ 0.6862,  0.3492, -0.6281]], requires_grad=True)
tensor([ 2.1960, -0.2132], requires_grad=True)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [22]:
def model(x):
    return x @ w.t() + b

In [24]:
preds = model(inputs_tensor)

In [25]:
print(preds)

tensor([[271.6121,  46.2660],
        [358.0963,  52.7605],
        [423.2386,  69.8473],
        [265.1209,  61.5529],
        [347.4388,  36.6898]], grad_fn=<AddBackward0>)


In [26]:
print(targets_tensor)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [27]:
# defining loss MSE
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff*diff)/diff.numel()

In [28]:
# Compute loss
loss = mse(preds,targets_tensor)
print(loss)

tensor(34894.6562, grad_fn=<DivBackward0>)


In [29]:
## Computing gradient

loss.backward()

In [32]:
print(w.grad)
print(w)

print(b.grad)
print(b)

tensor([[21817.7637, 22703.7578, 14151.5029],
        [-2958.3330, -4228.7432, -2524.7998]])
tensor([[ 1.4280,  1.6965,  1.1977],
        [ 0.6862,  0.3492, -0.6281]], requires_grad=True)
tensor([256.9013, -38.5767])
tensor([ 2.1960, -0.2132], requires_grad=True)


## Adjusting weights and bias using Gradient descent

In [33]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [34]:
print(w)

tensor([[ 1.2098,  1.4695,  1.0562],
        [ 0.7158,  0.3915, -0.6028]], requires_grad=True)


In [36]:
# Calculate loss
preds = model(inputs_tensor)
loss = mse(preds, targets_tensor)
print(loss)

tensor(23742.5840, grad_fn=<DivBackward0>)


In [38]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs_tensor)
    loss = mse(preds, targets_tensor)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

## Using Pytorch inbuilt fn

In [44]:
# Imports
import torch.nn as nn

In [45]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [46]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [47]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [48]:
train_ds = TensorDataset(inputs,targets)

In [49]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [50]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 91.,  88.,  64.],
         [ 69.,  96.,  70.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 91.,  88.,  64.]]),
 tensor([[ 81., 101.],
         [103., 119.],
         [ 22.,  37.],
         [103., 119.],
         [ 81., 101.]])]

In [51]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2679, -0.1036,  0.1474],
        [-0.0370,  0.1352,  0.4474]], requires_grad=True)
Parameter containing:
tensor([0.1090, 0.4654], requires_grad=True)


In [52]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [54]:
# Import nn.functional
import torch.nn.functional as F

loss_fn = F.mse_loss

print(loss_fn(preds,targets_tensor))

tensor(258.6304, grad_fn=<MseLossBackward>)


In [57]:
def fit(n_epochs,model,loss_fn,opt):
    for epoch in range(n_epochs):
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred,yb)
            loss.backward()
            # gradient descent
            opt.step()
            opt.zero_grad()
    print("Final loss",loss_fn(model(inputs), targets))

In [58]:
fit(100,model,loss_fn,opt)

Final loss tensor(16.3908, grad_fn=<MseLossBackward>)


## Using NN module

In [60]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3,3)
        self.act1 = nn.ReLU()
        self.l2 = nn.Linear(3,2)

    def forward(self,x):
        x = self.l1(x)
        x = self.act1(x)
        x = self.l2(x)
        return x

In [62]:
model = SimpleNet()
opt = torch.optim.SGD(model.parameters(), 1e-5)
loss_fn = F.mse_loss

In [63]:
fit(100,model,loss_fn,opt)

Final loss tensor(29.3251, grad_fn=<MseLossBackward>)
